In [ ]:
import duckdb 

con = duckdb.connect(database=':memory:')
con.execute(f"""INSTALL SPATIAL; LOAD SPATIAL; INSTALL HTTPS; LOAD HTTPFS""")

hist_bins = [0,1,2,3,4,5,6,7,8,9,10]

# Create temporary tables with spatial indexes
con.execute("""
CREATE TEMP TABLE temp_buildings AS 
SELECT geometry, risk_2011, risk_2047, wind_risk_2011, wind_risk_2047
FROM read_parquet('s3://carbonplan-ocr/intermediate/fire-risk/vector/prod/consolidated_geoparquet.parquet')
""")

con.execute("""
CREATE TEMP TABLE temp_counties AS 
SELECT NAME, geometry 
FROM read_parquet('s3://carbonplan-ocr/input/fire-risk/vector/aggregated_regions/counties.parquet')
""")

# Create spatial indexes on both tables
con.execute("CREATE INDEX buildings_spatial_idx ON temp_buildings USING RTREE (geometry)")
con.execute("CREATE INDEX counties_spatial_idx ON temp_counties USING RTREE (geometry)")


result = con.query(f"""
SELECT b.NAME as county_name, count(b.NAME) as building_count, 
avg(a.risk_2011) as fire_risk_2011_avg,
avg(a.risk_2047) as fire_risk_2047_avg,
avg(a.wind_risk_2011) as fire_wind_risk_2011_avg,
avg(a.wind_risk_2047) as fire_wind_risk_2047_avg,

histogram(a.risk_2011, {hist_bins}) as risk_2011,
histogram(a.risk_2047, {hist_bins}) as risk_2047,
histogram(a.wind_risk_2011, {hist_bins}) as wind_risk_2011,
histogram(a.wind_risk_2047, {hist_bins}) as wind_risk_2047,

FROM temp_buildings a
JOIN temp_counties b ON ST_Intersects(a.geometry, b.geometry) 
GROUP BY b.NAME 
""")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

OutOfMemoryException: Out of Memory Error: failed to offload data block of size 256.0 KiB (21.9 GiB/21.9 GiB used).
This limit was set by the 'max_temp_directory_size' setting.
By default, this setting utilizes the available disk space on the drive where the 'temp_directory' is located.
You can adjust this setting, by using (for example) PRAGMA max_temp_directory_size='10GiB'

Possible solutions:
* Reducing the number of threads (SET threads=X)
* Disabling insertion-order preservation (SET preserve_insertion_order=false)
* Increasing the memory limit (SET memory_limit='...GB')

See also https://duckdb.org/docs/stable/guides/performance/how_to_tune_workloads

In [ ]:
# try for a single county
# do a bbox subset for the buildings

In [ ]:
duckdb.sql("""DESCRIBE SELECT * FROM 's3://carbonplan-ocr/intermediate/fire-risk/vector/prod/consolidated_geoparquet.parquet'""")

In [ ]:
hist_bins = [0,1,2,3,4,5,6,7,8,9,10]
# duckdb.sql(f""" SELECT histogram(a.risk_2011, {hist_bins}) as risk_hist FROM 's3://carbonplan-ocr/intermediate/fire-risk/vector/prod/consolidated_geoparquet.parquet' as a""")



con.query(f"""
SELECT count(b.NAME) AS CNT, b.NAME, histogram(a.risk_2011, {hist_bins}) as risk_hist
FROM read_parquet('s3://carbonplan-ocr/intermediate/fire-risk/vector/prod/consolidated_geoparquet.parquet') a
JOIN read_parquet('s3://carbonplan-ocr/input/fire-risk/vector/aggregated_regions/counties.parquet')  b ON ST_Intersects(a.geometry, b.geometry) GROUP BY b.NAME """)



In [ ]:
# note: county names are not unique
duckdb.sql("""SELECT * FROM 's3://carbonplan-ocr/input/fire-risk/vector/aggregated_regions/counties.parquet' WHERE NAME == 'Missoula'""")

In [ ]:

# con.execute(f"""CREATE TABLE buildings AS SELECT geometry, bbox FROM read_parquet('s3://carbonplan-ocr/intermediate/fire-risk/vector/prod/consolidated_geoparquet.parquet')""")
# con.execute(f"""CREATE TABLE counties AS SELECT NAME, geometry FROM read_parquet('s3://carbonplan-ocr/input/fire-risk/vector/aggregated_regions/counties.parquet') WHERE NAME == 'Missoula'""")
# con.execute("""CREATE INDEX my_idx ON buildings USING RTREE (geometry);""")

In [ ]:
con.query("""
SELECT count(b.NAME) AS CNT, b.NAME
FROM read_parquet('s3://carbonplan-ocr/intermediate/fire-risk/vector/prod/consolidated_geoparquet.parquet') a
JOIN read_parquet('s3://carbonplan-ocr/input/fire-risk/vector/aggregated_regions/counties.parquet')  b ON ST_Intersects(a.geometry, b.geometry) GROUP BY b.NAME ORDER BY CNT """)

In [ ]:
con.query("""SELECT * FROM buildings_with_county""")

In [ ]:
con.query(""" SELECT 
        NAME,
        COUNT(*) as building_count
    FROM buildings_with_county
    GROUP BY NAME

""")

In [ ]:


's3://carbonplan-ocr/input/fire-risk/vector/aggregated_regions/counties.parquet',


In [ ]:


-- Read the geoparquet files
CREATE TABLE counties AS 
SELECT * FROM read_parquet('path/to/county_polygons.parquet');

CREATE TABLE buildings AS 
SELECT * FROM read_parquet('path/to/building_polygons.parquet');

-- Perform spatial join to assign buildings to counties
CREATE TABLE buildings_with_county AS
SELECT 
    b.*,
    c.county_name,
    c.state_code,
    c.county_fips
FROM buildings b
JOIN counties c ON ST_Intersects(b.geometry, c.geometry);

-- Method 1: Using histogram() with custom boundaries for fixed bins (0-10, 10-20, etc.)
WITH county_histograms AS (
    SELECT 
        county_name,
        state_code,
        COUNT(*) as building_count,
        AVG(risk1) as risk1_avg,
        AVG(risk2) as risk2_avg,
        
        -- Create histograms with fixed 10-unit bins (0-10, 10-20, ..., 90-100)
        histogram(risk1, [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]) as risk1_hist,
        histogram(risk2, [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]) as risk2_hist
    FROM buildings_with_county
    WHERE risk1 IS NOT NULL AND risk2 IS NOT NULL
    GROUP BY county_name, state_code
)
SELECT 
    county_name,
    state_code,
    building_count,
    risk1_avg,
    risk2_avg,
    
    -- Convert histogram maps to your desired bin structure
    STRUCT_PACK(
        bin_0_10 := COALESCE(risk1_hist[10], 0),
        bin_10_20 := COALESCE(risk1_hist[20], 0),
        bin_20_30 := COALESCE(risk1_hist[30], 0),
        bin_30_40 := COALESCE(risk1_hist[40], 0),
        bin_40_50 := COALESCE(risk1_hist[50], 0),
        bin_50_60 := COALESCE(risk1_hist[60], 0),
        bin_60_70 := COALESCE(risk1_hist[70], 0),
        bin_70_80 := COALESCE(risk1_hist[80], 0),
        bin_80_90 := COALESCE(risk1_hist[90], 0),
        bin_90_100 := COALESCE(risk1_hist[100], 0)
    ) as risk1_bins,
    
    STRUCT_PACK(
        bin_0_10 := COALESCE(risk2_hist[10], 0),
        bin_10_20 := COALESCE(risk2_hist[20], 0),
        bin_20_30 := COALESCE(risk2_hist[30], 0),
        bin_30_40 := COALESCE(risk2_hist[40], 0),
        bin_40_50 := COALESCE(risk2_hist[50], 0),
        bin_50_60 := COALESCE(risk2_hist[60], 0),
        bin_60_70 := COALESCE(risk2_hist[70], 0),
        bin_70_80 := COALESCE(risk2_hist[80], 0),
        bin_80_90 := COALESCE(risk2_hist[90], 0),
        bin_90_100 := COALESCE(risk2_hist[100], 0)
    ) as risk2_bins
FROM county_histograms;

-- Method 2: Create the exact nested structure you want using STRUCT_PACK
WITH county_stats AS (
    SELECT 
        county_name,
        state_code,
        COUNT(*) as building_count,
        AVG(risk1) as risk1_avg,
        AVG(risk2) as risk2_avg,
        histogram(risk1, [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]) as risk1_hist,
        histogram(risk2, [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]) as risk2_hist
    FROM buildings_with_county
    WHERE risk1 IS NOT NULL AND risk2 IS NOT NULL
    GROUP BY county_name, state_code
)
SELECT 
    STRUCT_PACK(
        county_name := county_name,
        state_code := state_code,
        building_count := building_count,
        risk1_avg := risk1_avg,
        risk2_avg := risk2_avg,
        bins := STRUCT_PACK(
            risk1 := STRUCT_PACK(
                bin_0_10 := COALESCE(risk1_hist[10], 0),
                bin_10_20 := COALESCE(risk1_hist[20], 0),
                bin_20_30 := COALESCE(risk1_hist[30], 0),
                bin_30_40 := COALESCE(risk1_hist[40], 0),
                bin_40_50 := COALESCE(risk1_hist[50], 0),
                bin_50_60 := COALESCE(risk1_hist[60], 0),
                bin_60_70 := COALESCE(risk1_hist[70], 0),
                bin_70_80 := COALESCE(risk1_hist[80], 0),
                bin_80_90 := COALESCE(risk1_hist[90], 0),
                bin_90_100 := COALESCE(risk1_hist[100], 0)
            ),
            risk2 := STRUCT_PACK(
                bin_0_10 := COALESCE(risk2_hist[10], 0),
                bin_10_20 := COALESCE(risk2_hist[20], 0),
                bin_20_30 := COALESCE(risk2_hist[30], 0),
                bin_30_40 := COALESCE(risk2_hist[40], 0),
                bin_40_50 := COALESCE(risk2_hist[50], 0),
                bin_50_60 := COALESCE(risk2_hist[60], 0),
                bin_60_70 := COALESCE(risk2_hist[70], 0),
                bin_70_80 := COALESCE(risk2_hist[80], 0),
                bin_80_90 := COALESCE(risk2_hist[90], 0),
                bin_90_100 := COALESCE(risk2_hist[100], 0)
            )
        )
    ) as county_data
FROM county_stats;

-- Method 3: Using histogram_values for more dynamic binning
-- Step 1: Create summary stats table
CREATE TABLE county_building_stats AS
SELECT 
    county_name,
    state_code,
    county_fips,
    COUNT(*) as building_count,
    AVG(risk1) as risk1_avg,
    AVG(risk2) as risk2_avg,
    histogram_values(buildings_with_county, 'risk1', 'bin_count', 10) as risk1_bins_detail,
    histogram_values(buildings_with_county, 'risk2', 'bin_count', 10) as risk2_bins_detail
FROM buildings_with_county
WHERE risk1 IS NOT NULL AND risk2 IS NOT NULL
GROUP BY county_name, state_code, county_fips;

-- Step 2: Join back to original county polygons to preserve geometry
CREATE TABLE counties_with_stats AS
SELECT 
    c.*,  -- All original county columns including geometry
    COALESCE(cbs.building_count, 0) as building_count,
    cbs.risk1_avg,
    cbs.risk2_avg,
    cbs.risk1_bins_detail,
    cbs.risk2_bins_detail
FROM counties c
LEFT JOIN county_building_stats cbs 
    ON c.county_name = cbs.county_name 
    AND c.state_code = cbs.state_code;

-- Step 3: Export as GeoParquet
COPY counties_with_stats TO 'counties_with_building_stats.parquet' (FORMAT PARQUET);

-- Step 4: Alternative - Export as FlatGeobuf
-- First install the spatial extension if not already done
-- INSTALL spatial;
-- LOAD spatial;
COPY counties_with_stats TO 'counties_with_building_stats.fgb' (FORMAT GDAL, DRIVER 'FlatGeobuf');

-- Method 4: Export as JSON for easy consumption
COPY (
    WITH county_stats AS (
        SELECT 
            county_name,
            state_code,
            COUNT(*) as building_count,
            AVG(risk1) as risk1_avg,
            AVG(risk2) as risk2_avg,
            histogram(risk1, [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]) as risk1_hist,
            histogram(risk2, [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]) as risk2_hist
        FROM buildings_with_county
        WHERE risk1 IS NOT NULL AND risk2 IS NOT NULL
        GROUP BY county_name, state_code
    )
    SELECT 
        STRUCT_PACK(
            county_name := county_name,
            state_code := state_code,
            building_count := building_count,
            risk1_avg := risk1_avg,
            risk2_avg := risk2_avg,
            bins := STRUCT_PACK(
                risk1 := STRUCT_PACK(
                    bin_0_10 := COALESCE(risk1_hist[10], 0),
                    bin_10_20 := COALESCE(risk1_hist[20], 0),
                    bin_20_30 := COALESCE(risk1_hist[30], 0),
                    bin_30_40 := COALESCE(risk1_hist[40], 0),
                    bin_40_50 := COALESCE(risk1_hist[50], 0),
                    bin_50_60 := COALESCE(risk1_hist[60], 0),
                    bin_60_70 := COALESCE(risk1_hist[70], 0),
                    bin_70_80 := COALESCE(risk1_hist[80], 0),
                    bin_80_90 := COALESCE(risk1_hist[90], 0),
                    bin_90_100 := COALESCE(risk1_hist[100], 0)
                ),
                risk2 := STRUCT_PACK(
                    bin_0_10 := COALESCE(risk2_hist[10], 0),
                    bin_10_20 := COALESCE(risk2_hist[20], 0),
                    bin_20_30 := COALESCE(risk2_hist[30], 0),
                    bin_30_40 := COALESCE(risk2_hist[40], 0),
                    bin_40_50 := COALESCE(risk2_hist[50], 0),
                    bin_50_60 := COALESCE(risk2_hist[60], 0),
                    bin_60_70 := COALESCE(risk2_hist[70], 0),
                    bin_70_80 := COALESCE(risk2_hist[80], 0),
                    bin_80_90 := COALESCE(risk2_hist[90], 0),
                    bin_90_100 := COALESCE(risk2_hist[100], 0)
                )
            )
        ) as county_data
    FROM county_stats
) TO 'county_histograms.json' (FORMAT JSON, ARRAY true);